<a href="https://colab.research.google.com/github/wikibook/machine-learning/blob/2.0/jupyter_notebook/6.2_RNN_기초_텐서플로우_실습.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이번 실습을 통해, 텐서플로우로 구현된 RNN을 직접 탐구해보도록 하겠습니다.

In [30]:
from IPython.display import Image

기본적으로 RNN은 아래의 다이어그램과 같이 구현됩니다.  
보시다시피, RNN은 입력값(x), 출력값(output), 상태값(state), 가중치(W), 편향값(b), 그리고 활성화함수(tanh)로 구성되어 있습니다.

In [31]:
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/simple_rnn.png", width=450, height=200)

# RNN 텐서플로우 코드 구현
위에서 다뤄보았던 예제를 직접 텐서플로우로 구현하여 입력값, 출력값, 상태값, 가중치, 편향값을 확인해보도록 하겠습니다.

In [32]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

In [33]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, SimpleRNN
from tensorflow.keras.models import Model
import numpy as np

# 항상 같은 결과를 갖기 위해 랜덤 시드 설정
tf.random.set_seed(1)
np.random.seed(1)

In [34]:
# 입력값의 형태를 지정합니다.
inputs = Input(shape=(1,2))
"""
RNN 셀의 속성을 지정합니다.
3: 3차원 벡터의 출력값 지정합니다.
return_state=True: RNN 셀의 상태값(state)를 출력하도록 지정합니다.
"""
output, state = SimpleRNN(3, return_state=True)(inputs)
model = Model(inputs=inputs, outputs=[output, state])

In [35]:
# 테스트 입력값
data = np.array([[ [1,2] ]])
# 출력값, 상태값 프린트
output, state = model.predict(data)
print("output: ",output)
print("state: ",state)

output:  [[-0.77005637 -0.03206512  0.72178566]]
state:  [[-0.77005637 -0.03206512  0.72178566]]


위 코드의 출력값을 통해, 텐서플로우 RNN이 우리가 이론으로 다룬 RNN과 같은 것을 확인할 수 있습니다..  
1. RNN 셀이 한개일 경우 그 출력값과 상태값이 동일합니다.  

In [36]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/rnn_single.png", width=450, height=200)

In [37]:
# 입력값에 대한 가중치 행렬의 값을 확인합니다.
model.layers[1].weights[0]

<tf.Variable 'simple_rnn_4/kernel:0' shape=(2, 3) dtype=float32, numpy=
array([[-0.73366153,  0.8796015 ,  0.28695   ],
       [-0.14340228, -0.4558388 ,  0.3122064 ]], dtype=float32)>

입력값이 1x2 행렬이므로 입력값에 대한 가중치 행렬은 2x3 행렬입니다.

In [38]:
# 이전 상태값에 대한 가중치 행렬의 값을 확인합니다.
model.layers[1].weights[1]

<tf.Variable 'simple_rnn_4/recurrent_kernel:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.2532742 , -0.8955574 ,  0.36582667],
       [ 0.8398052 ,  0.0158366 , -0.5426569 ],
       [-0.4801869 , -0.4446641 , -0.7561047 ]], dtype=float32)>

상태값이 1x3 행렬이므로, 상태값에 대한 가중치 행렬은 3x3 행렬입니다.

In [39]:
# 편향값 행렬의 값을 확인합니다.
model.layers[1].weights[2]

<tf.Variable 'simple_rnn_4/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>

상태값이 3차원 벡터이므로, 3개의 편향값이 존재합니다.

# 텐서플로우로 단어 품사 구분하기 예제 구현하기
이론에서 다뤘던 "I work at google", "I google at work"의 단어 품사를 구분하는 RNN 코드를 텐서플로우로 구현해보도록 하겠습니다.  
각 단어는 one hot encoding으로 표현합니다.

In [40]:
# I      [1,0,0,0]
# work   [0,1,0,0]
# at     [0,0,1,0]
# google [0,0,0,1]
#
# I work at google =  [ [1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1] ]
# I google at work =  [ [1,0,0,0], [0,0,0,1], [0,0,1,0], [0,1,0,0] ]

data = np.array([
    [ [1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1] ],
    [ [1,0,0,0], [0,0,0,1], [0,0,1,0], [0,1,0,0] ]
])

아래의 코드를 실행하여, 결과값을 확인합니다.

In [41]:
# 입력값의 형태를 지정합니다.
inputs = Input(shape=(4, 4))
"""
RNN 셀의 속성을 지정합니다.
3: 3차원 벡터의 출력값 지정합니다.
return_state=True: RNN 셀의 상태값(state)를 출력하도록 지정합니다.
"""
output, state = SimpleRNN(3, return_state=True, return_sequences=True)(inputs)
model = Model(inputs=inputs, outputs=[output, state])

In [42]:
# 출력값, 상태값 프린트
output, state = model.predict(data)

In [43]:
print("I work at google: ",output[0])
print("I google at work: ",output[1])

I work at google:  [[ 0.54391825  0.24354757  0.02726344]
 [ 0.2543367  -0.22151536 -0.70710266]
 [-0.0213326   0.71822613  0.06801447]
 [-0.27135777 -0.4469267   0.00939073]]
I google at work:  [[ 0.54391825  0.24354757  0.02726344]
 [ 0.17483439 -0.34557    -0.5084784 ]
 [-0.20281316  0.7054854  -0.09443939]
 [-0.2546866  -0.15914649 -0.16518137]]


1. 두 문장의 첫단어 출력값이 동일합니다. 이는 첫단어에는 이전 상태값이 존재하지 않기 때문입니다.  
2. 두번째 단어부터의 출력값은 두 문장이 다르게 나타납니다. 이는 이전 상태값이 현재 출력값에 영향을 주기 때문입니다.

In [44]:
print("I work at google: state: ",state[0])
print("I google at work: state: ",state[1])

I work at google: state:  [-0.27135777 -0.4469267   0.00939073]
I google at work: state:  [-0.2546866  -0.15914649 -0.16518137]


3. 상태값은 항상 마지막 단어의 출력값과 동일합니다.  